<a href="https://colab.research.google.com/github/rahatarinasir/Dayche/blob/master/Keras_implementation_of_neural_networks_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialization

In [ ]:
import keras
import numpy as np 
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf 
from keras.layers import Layer
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix

## Breast Cancer Data Loading

In [ ]:
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target
label = np.zeros((len(y), 2))
label[np.where(y == 1), 1] = 1
label[np.where(y == 0), 0] = 1

Xtrain, Xtest, y_train, y_test = train_test_split(X, label, test_size = 0.3)
scaler_X = MinMaxScaler().fit(Xtrain)
X_train, X_test = scaler_X.transform(Xtrain), scaler_X.transform(Xtest)

## Define RBF Layer

In [ ]:
class RBFLayer(Layer):
    
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)

    def build(self, input_shape):
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform', 
                                  trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff,2), axis=1)
        res = K.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

## Define Rough Layer

In [ ]:
class RoghLayer(Layer):
    def __init__(self, units, alpha = 0.5, beta = 0.5, activation = None, **kwargs):
        super(RoghLayer, self).__init__(**kwargs)
        self.units = units
        self.activation = activation
        self.alpha = alpha
        self.beta = beta
        assert (self.alpha + self.beta == 1)

    def build(self, input_shape):
        self.wl = self.add_weight(name='lower_weights',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        self.bl = self.add_weight(name='lower_bias',
                                  shape=(1, self.units),
                                  initializer='uniform',
                                  trainable=True)
        self.wu = self.add_weight(name='upper_weights',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        self.bu = self.add_weight(name='upper_bias',
                                  shape=(1, self.units),
                                  initializer='uniform',
                                  trainable=True)
            
        super(RoghLayer, self).build(input_shape)

    def call(self, inputs):
        net_l = tf.matmul(inputs, self.wl) + self.bl
        net_u = tf.matmul(inputs, self.wu) + self.bu
        if self.activation == None:
            ol = netl
            ou = netu
        elif self.activation == 'sigmoid':
            ol = tf.keras.activations.sigmoid(net_l)
            ou = tf.keras.activations.sigmoid(net_u)
        elif self.activation == 'tanh':
            ol = tf.keras.activations.tanh(net_l)
            ou = tf.keras.activations.tanh(net_u)
        elif self.activation == 'relu':
            ol = tf.keras.activations.relu(net_l)
            ou = tf.keras.activations.relu(net_u)
    
        o = self.alpha * ol + self.beta * ou
        return o

## Define Flexible Layer

In [ ]:
class FlexibleLayer(Layer):

    def __init__(self, units, activation=None, **kwargs):
        super(FlexibleLayer, self).__init__(**kwargs)
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        self.w = self.add_weight(name='weights',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        self.b = self.add_weight(name='bias',
                                  shape=(1, self.units),
                                  initializer='uniform',
                                  trainable=True)
        self.g = self.add_weight(name='flexible_parameter',
                                 shape=(1, self.units),
                                 initializer='uniform',
                                 trainable=True)

        super(FlexibleLayer, self).build(input_shape)


    def call(self, inputs):
        net = tf.matmul(inputs, self.w) + self.b
        if self.activation == None:
            o = net
        elif self.activation == 'sigmoid':
            net = tf.math.multiply(tf.math.abs(self.g), net)
            o = tf.keras.activations.sigmoid(net)
        elif self.activation == 'tanh':
            net = tf.math.multiply(tf.math.abs(self.g), net)
            o = tf.keras.activations.tanh(net)

        return o

## Simple MLP Training

In [ ]:
model = Sequential()
model.add(Dense(50, activation = 'sigmoid'))
model.add(Dense(2, activation = 'sigmoid'))
opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer = opt, loss=loss, metrics = 'binary_crossentropy')
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100, batch_size = 5)

Epoch 1/100
80/80 [==============================] - 27s 11ms/step - loss: 0.7021 - binary_crossentropy: 0.7021 - val_loss: 0.6634 - val_binary_crossentropy: 0.6634
Epoch 2/100
80/80 [==============================] - 0s 2ms/step - loss: 0.6552 - binary_crossentropy: 0.6552 - val_loss: 0.6580 - val_binary_crossentropy: 0.6580
Epoch 3/100
80/80 [==============================] - 0s 2ms/step - loss: 0.6274 - binary_crossentropy: 0.6274 - val_loss: 0.6541 - val_binary_crossentropy: 0.6541
Epoch 4/100
80/80 [==============================] - 0s 2ms/step - loss: 0.6416 - binary_crossentropy: 0.6416 - val_loss: 0.6488 - val_binary_crossentropy: 0.6488
Epoch 5/100
80/80 [==============================] - 0s 2ms/step - loss: 0.6306 - binary_crossentropy: 0.6306 - val_loss: 0.6439 - val_binary_crossentropy: 0.6439
Epoch 6/100
80/80 [==============================] - 0s 2ms/step - loss: 0.6274 - binary_crossentropy: 0.6274 - val_loss: 0.6389 - val_binary_crossentropy: 0.6389
Epoch 7/100
80/80 [=

Epoch 51/100
80/80 [==============================] - 0s 3ms/step - loss: 0.3650 - binary_crossentropy: 0.3650 - val_loss: 0.3898 - val_binary_crossentropy: 0.3898
Epoch 52/100
80/80 [==============================] - 0s 4ms/step - loss: 0.3756 - binary_crossentropy: 0.3756 - val_loss: 0.3852 - val_binary_crossentropy: 0.3852
Epoch 53/100
80/80 [==============================] - 0s 4ms/step - loss: 0.3633 - binary_crossentropy: 0.3633 - val_loss: 0.3797 - val_binary_crossentropy: 0.3797
Epoch 54/100
80/80 [==============================] - 0s 5ms/step - loss: 0.3411 - binary_crossentropy: 0.3411 - val_loss: 0.3751 - val_binary_crossentropy: 0.3751
Epoch 55/100
80/80 [==============================] - 0s 4ms/step - loss: 0.3577 - binary_crossentropy: 0.3577 - val_loss: 0.3708 - val_binary_crossentropy: 0.3708
Epoch 56/100
80/80 [==============================] - 0s 3ms/step - loss: 0.3724 - binary_crossentropy: 0.3724 - val_loss: 0.3666 - val_binary_crossentropy: 0.3666
Epoch 57/100
80/

### Evaluation

In [ ]:
y_pre = model.predict(X_test)
confusion_matrix(np.argmax(y_pre, axis=1), np.argmax(y_test, axis = 1))

array([[ 60,   1],
       [  7, 103]], dtype=int64)

## RBF Neural Netrowk Training

In [ ]:
model = Sequential()
model.add(RBFLayer(50, 0.15, name = 'RBF', trainable = True))
model.add(keras.layers.Dense(2, activation = 'sigmoid'))
opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer = opt, loss=loss, metrics = 'binary_crossentropy')
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 200, batch_size = 5)

Epoch 1/200
80/80 [==============================] - 4s 6ms/step - loss: 0.6150 - binary_crossentropy: 0.6150 - val_loss: 0.6138 - val_binary_crossentropy: 0.6138
Epoch 2/200
80/80 [==============================] - 0s 2ms/step - loss: 0.5316 - binary_crossentropy: 0.5316 - val_loss: 0.6077 - val_binary_crossentropy: 0.6077
Epoch 3/200
80/80 [==============================] - 0s 2ms/step - loss: 0.4759 - binary_crossentropy: 0.4759 - val_loss: 0.5616 - val_binary_crossentropy: 0.5616
Epoch 4/200
80/80 [==============================] - 0s 2ms/step - loss: 0.5112 - binary_crossentropy: 0.5112 - val_loss: 0.5722 - val_binary_crossentropy: 0.5722
Epoch 5/200
80/80 [==============================] - 0s 2ms/step - loss: 0.4753 - binary_crossentropy: 0.4753 - val_loss: 0.5116 - val_binary_crossentropy: 0.5116
Epoch 6/200
80/80 [==============================] - 0s 2ms/step - loss: 0.4408 - binary_crossentropy: 0.4408 - val_loss: 0.4931 - val_binary_crossentropy: 0.4931
Epoch 7/200
80/80 [===

80/80 [==============================] - 0s 2ms/step - loss: 0.0886 - binary_crossentropy: 0.0886 - val_loss: 0.1285 - val_binary_crossentropy: 0.1285
Epoch 52/200
80/80 [==============================] - 0s 2ms/step - loss: 0.1211 - binary_crossentropy: 0.1211 - val_loss: 0.1246 - val_binary_crossentropy: 0.1246
Epoch 53/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0975 - binary_crossentropy: 0.0975 - val_loss: 0.1234 - val_binary_crossentropy: 0.1234
Epoch 54/200
80/80 [==============================] - 0s 2ms/step - loss: 0.1107 - binary_crossentropy: 0.1107 - val_loss: 0.1230 - val_binary_crossentropy: 0.1230
Epoch 55/200
80/80 [==============================] - 0s 2ms/step - loss: 0.1210 - binary_crossentropy: 0.1210 - val_loss: 0.1259 - val_binary_crossentropy: 0.1259
Epoch 56/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0983 - binary_crossentropy: 0.0983 - val_loss: 0.1211 - val_binary_crossentropy: 0.1211
Epoch 57/200
80/80 [=========

80/80 [==============================] - 0s 2ms/step - loss: 0.0833 - binary_crossentropy: 0.0833 - val_loss: 0.0931 - val_binary_crossentropy: 0.0931
Epoch 102/200
80/80 [==============================] - 0s 3ms/step - loss: 0.1086 - binary_crossentropy: 0.1086 - val_loss: 0.0937 - val_binary_crossentropy: 0.0937
Epoch 103/200
80/80 [==============================] - 0s 4ms/step - loss: 0.0900 - binary_crossentropy: 0.0900 - val_loss: 0.0915 - val_binary_crossentropy: 0.0915
Epoch 104/200
80/80 [==============================] - 0s 3ms/step - loss: 0.0770 - binary_crossentropy: 0.0770 - val_loss: 0.0901 - val_binary_crossentropy: 0.0901
Epoch 105/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0765 - binary_crossentropy: 0.0765 - val_loss: 0.0932 - val_binary_crossentropy: 0.0932
Epoch 106/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0770 - binary_crossentropy: 0.0770 - val_loss: 0.0894 - val_binary_crossentropy: 0.0894
Epoch 107/200
80/80 [===

Epoch 151/200
80/80 [==============================] - 0s 2ms/step - loss: 0.1145 - binary_crossentropy: 0.1145 - val_loss: 0.0818 - val_binary_crossentropy: 0.0818
Epoch 152/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0885 - binary_crossentropy: 0.0885 - val_loss: 0.0774 - val_binary_crossentropy: 0.0774
Epoch 153/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0488 - binary_crossentropy: 0.0488 - val_loss: 0.0797 - val_binary_crossentropy: 0.0797
Epoch 154/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0791 - binary_crossentropy: 0.0791 - val_loss: 0.0772 - val_binary_crossentropy: 0.0772
Epoch 155/200
80/80 [==============================] - 0s 2ms/step - loss: 0.1051 - binary_crossentropy: 0.1051 - val_loss: 0.0799 - val_binary_crossentropy: 0.0799
Epoch 156/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0627 - binary_crossentropy: 0.0627 - val_loss: 0.0774 - val_binary_crossentropy: 0.0774
Epoch 157/

### Evaluation

In [ ]:
y_pre = model.predict(X_test)
confusion_matrix(np.argmax(y_pre, axis=1), np.argmax(y_test, axis = 1))

array([[ 55,   2],
       [  4, 110]], dtype=int64)

## Rough Neural Network Training 

In [ ]:
model = Sequential()
model.add(RoghLayer(50, activation = 'sigmoid', name = 'Rough'))
model.add(keras.layers.Dense(2, activation = 'sigmoid'))
model.compile(optimizer = opt, loss=loss, metrics = 'binary_crossentropy')
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100, batch_size = 5)

Epoch 1/100
80/80 [==============================] - 1s 2ms/step - loss: 0.6824 - binary_crossentropy: 0.6824 - val_loss: 0.6331 - val_binary_crossentropy: 0.6331
Epoch 2/100
80/80 [==============================] - 0s 1ms/step - loss: 0.6478 - binary_crossentropy: 0.6478 - val_loss: 0.6203 - val_binary_crossentropy: 0.6203
Epoch 3/100
80/80 [==============================] - 0s 1ms/step - loss: 0.6290 - binary_crossentropy: 0.6290 - val_loss: 0.6229 - val_binary_crossentropy: 0.6229
Epoch 4/100
80/80 [==============================] - 0s 1ms/step - loss: 0.6063 - binary_crossentropy: 0.6063 - val_loss: 0.5809 - val_binary_crossentropy: 0.5809
Epoch 5/100
80/80 [==============================] - 0s 1ms/step - loss: 0.5737 - binary_crossentropy: 0.5737 - val_loss: 0.5446 - val_binary_crossentropy: 0.5446
Epoch 6/100
80/80 [==============================] - 0s 1ms/step - loss: 0.5370 - binary_crossentropy: 0.5370 - val_loss: 0.5078 - val_binary_crossentropy: 0.5078
Epoch 7/100
80/80 [===

80/80 [==============================] - 0s 1ms/step - loss: 0.1109 - binary_crossentropy: 0.1109 - val_loss: 0.1213 - val_binary_crossentropy: 0.1213
Epoch 52/100
80/80 [==============================] - 0s 2ms/step - loss: 0.1091 - binary_crossentropy: 0.1091 - val_loss: 0.1242 - val_binary_crossentropy: 0.1242
Epoch 53/100
80/80 [==============================] - 0s 2ms/step - loss: 0.1093 - binary_crossentropy: 0.1093 - val_loss: 0.1189 - val_binary_crossentropy: 0.1189
Epoch 54/100
80/80 [==============================] - 0s 1ms/step - loss: 0.1073 - binary_crossentropy: 0.1073 - val_loss: 0.1195 - val_binary_crossentropy: 0.1195
Epoch 55/100
80/80 [==============================] - 0s 1ms/step - loss: 0.1050 - binary_crossentropy: 0.1050 - val_loss: 0.1183 - val_binary_crossentropy: 0.1183
Epoch 56/100
80/80 [==============================] - 0s 2ms/step - loss: 0.1055 - binary_crossentropy: 0.1055 - val_loss: 0.1159 - val_binary_crossentropy: 0.1159
Epoch 57/100
80/80 [=========

### Evaluation

In [ ]:
y_pre = model.predict(X_test)
confusion_matrix(np.argmax(y_pre, axis=1), np.argmax(y_test, axis = 1))

array([[ 55,   0],
       [  4, 112]], dtype=int64)

## Flexible Neural Network Training

In [ ]:
model = Sequential()
model.add(FlexibleLayer(50, activation = 'sigmoid', name = 'Flexible'))
model.add(keras.layers.Dense(2, activation = 'sigmoid'))
model.compile(optimizer = opt, loss=loss, metrics = 'binary_crossentropy')
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100, batch_size = 5)

Epoch 1/100
80/80 [==============================] - 1s 2ms/step - loss: 0.6838 - binary_crossentropy: 0.6838 - val_loss: 0.6460 - val_binary_crossentropy: 0.6460
Epoch 2/100
80/80 [==============================] - 0s 1ms/step - loss: 0.6718 - binary_crossentropy: 0.6718 - val_loss: 0.6444 - val_binary_crossentropy: 0.6444
Epoch 3/100
80/80 [==============================] - 0s 1ms/step - loss: 0.6726 - binary_crossentropy: 0.6726 - val_loss: 0.6460 - val_binary_crossentropy: 0.6460
Epoch 4/100
80/80 [==============================] - 0s 1ms/step - loss: 0.6728 - binary_crossentropy: 0.6728 - val_loss: 0.6599 - val_binary_crossentropy: 0.6599
Epoch 5/100
80/80 [==============================] - 0s 1ms/step - loss: 0.6812 - binary_crossentropy: 0.6812 - val_loss: 0.6443 - val_binary_crossentropy: 0.6443
Epoch 6/100
80/80 [==============================] - 0s 1ms/step - loss: 0.6710 - binary_crossentropy: 0.6710 - val_loss: 0.6440 - val_binary_crossentropy: 0.6440
Epoch 7/100
80/80 [===

80/80 [==============================] - 0s 1ms/step - loss: 0.0844 - binary_crossentropy: 0.0844 - val_loss: 0.0956 - val_binary_crossentropy: 0.0956
Epoch 52/100
80/80 [==============================] - 0s 1ms/step - loss: 0.0852 - binary_crossentropy: 0.0852 - val_loss: 0.0997 - val_binary_crossentropy: 0.0997
Epoch 53/100
80/80 [==============================] - 0s 1ms/step - loss: 0.0767 - binary_crossentropy: 0.0767 - val_loss: 0.1029 - val_binary_crossentropy: 0.1029
Epoch 54/100
80/80 [==============================] - 0s 1ms/step - loss: 0.0798 - binary_crossentropy: 0.0798 - val_loss: 0.0932 - val_binary_crossentropy: 0.0932
Epoch 55/100
80/80 [==============================] - 0s 1ms/step - loss: 0.0759 - binary_crossentropy: 0.0759 - val_loss: 0.0852 - val_binary_crossentropy: 0.0852
Epoch 56/100
80/80 [==============================] - 0s 1ms/step - loss: 0.0779 - binary_crossentropy: 0.0779 - val_loss: 0.0832 - val_binary_crossentropy: 0.0832
Epoch 57/100
80/80 [=========

### Evaluation

In [ ]:
y_pre = model.predict(X_test)
confusion_matrix(np.argmax(y_pre, axis=1), np.argmax(y_test, axis = 1))

array([[ 56,   0],
       [  3, 112]], dtype=int64)

In [ ]:
model.weights

[<tf.Variable 'Flexible/weights:0' shape=(30, 50) dtype=float32, numpy=
 array([[-0.01715174,  0.0243917 ,  0.07103336, ..., -0.04188572,
          0.00655605,  0.04198056],
        [ 0.03261117, -0.05357175,  0.01264675, ...,  0.03203963,
         -0.04531412,  0.0451592 ],
        [ 0.03131215, -0.03759532,  0.01214976, ..., -0.00590589,
          0.04022065, -0.00754653],
        ...,
        [ 0.03841779, -0.05368353,  0.02653261, ...,  0.02741181,
         -0.02176145, -0.01916105],
        [-0.03964899,  0.0342703 ,  0.03903956, ..., -0.01459446,
         -0.022578  ,  0.02915992],
        [ 0.02054088, -0.02397787,  0.03144042, ..., -0.00922068,
          0.03050829, -0.04144588]], dtype=float32)>,
 <tf.Variable 'Flexible/bias:0' shape=(1, 50) dtype=float32, numpy=
 array([[-3.4228705e-02,  7.6365019e-03,  3.9036095e-02, -7.0098197e-01,
         -3.4626567e-01, -3.3329248e-02, -1.3325995e-02, -1.4548036e-02,
          3.1278197e-02,  3.5660737e+00,  1.0301395e-02, -3.2516312e-02